In [1]:
import os
# os.chdir('../')
os.getcwd()

'C:\\Users\\頼仁\\Documents\\github\\TrAdaBoost2\\notebook'

In [2]:
import sys
# sys.path.append('../../scikit-learn/')
sys.path.append('../my-scikit-learn/')

In [3]:
import sklearn

In [4]:
import importlib
importlib.reload(sklearn) 

<module 'sklearn' from 'c:\\users\\頼仁\\documents\\github\\scikit-learn\\sklearn\\__init__.py'>

In [5]:
import pandas as pd
import numpy as np
from scipy.io import arff

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

In [6]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

In [7]:
def make_dataset_concrete():
    data = arff.loadarff('../dataset/transfer data/concrete1.arff')
    df1 = pd.DataFrame(data[0])
    df1["label"] = 0
    data = arff.loadarff('../dataset/transfer data/concrete2.arff')
    df2 = pd.DataFrame(data[0])
    df2["label"] = 1
    data = arff.loadarff('../dataset/transfer data/concrete3.arff')
    df3 = pd.DataFrame(data[0])
    df3["label"] = 2
    df = pd.concat([df1, df2, df3], sort=False).reset_index(drop=True)
    return df

In [8]:
df = make_dataset_concrete()
df

,b,c,d,e,f,g,h,i,label
0,153.0,0.0,192.0,0.0,887.0,942.0,3.0,4.57,0
1,153.0,0.0,192.0,0.0,887.0,942.0,90.0,25.46,0
2,153.0,0.0,192.0,0.0,887.0,942.0,7.0,7.68,0
3,153.0,0.0,192.0,0.0,887.0,942.0,28.0,17.28,0
4,162.4,0.0,203.5,0.0,938.2,849.0,28.0,20.59,0
...,...,...,...,...,...,...,...,...,...
1025,0.0,0.0,173.0,0.0,1125.0,613.0,28.0,67.31,2
1026,0.0,0.0,173.0,0.0,1125.0,613.0,90.0,69.66,2
1027,0.0,0.0,173.0,0.0,1125.0,613.0,180.0,71.62,2
1028,0.0,0.0,173.0,0.0,1125.0,613.0,270.0,74.17,2


In [9]:
n_target=25
y_feature="i"

In [10]:
learning_rate = 0.1
n_estimators = {'target': 30, 'all': 200, 'source':150}
max_depth = 4
validation_fraction= 0.2
n_iter_no_change= 5
tol= 0.1
random_state = 0

In [11]:
target_domain = 0

target_train = df.query('label==@target_domain').sample(n_target)
target_test = df.query('label==@target_domain').drop(index=target_train.index)
target_train_X = target_train.drop(columns=[y_feature, 'label'])
target_train_y = target_train[y_feature]
target_test_X = target_test.drop(columns=[y_feature, 'label']).values
target_test_y = target_test[y_feature].values
source_X = df.query('label!=@target_domain').drop(columns=[y_feature, 'label'])
source_y = df.query('label!=@target_domain')[y_feature]

X_train = pd.concat([source_X, target_train_X]).values
y_train = pd.concat([source_y, target_train_y]).values
sample_size = [source_X.shape[0], target_train_X.shape[0]]


# targetのみgradient boost
reg_target = GradientBoostingRegressor(
                    n_estimators = n_estimators['target'], 
                    learning_rate=learning_rate,
                    max_depth = max_depth,
                    random_state = random_state)
reg_target.fit(target_train_X.values, target_train_y.values)
y_pred_target = reg_target.predict(target_test_X)
rmse_target = mean_squared_error(target_test_y, y_pred_target)**0.5

# source+target gradient boost
reg_all = GradientBoostingRegressor(
                    n_estimators = n_estimators["all"], 
                    learning_rate=learning_rate,
                    max_depth = max_depth,
                    random_state = random_state)
reg_all.fit(X_train, y_train)
y_pred_all = reg_all.predict(target_test_X)
rmse_all = mean_squared_error(target_test_y, y_pred_all)**0.5

# sourceのみgradient boost
reg_source = GradientBoostingRegressor(
                    n_estimators = n_estimators['source'], 
                    learning_rate=learning_rate,
                    max_depth = max_depth,
                    random_state = random_state)
reg_source.fit(source_X, source_y)
y_pred_source = reg_source.predict(target_test_X)
rmse_source = mean_squared_error(target_test_y, y_pred_source)**0.5

print('gradient boost target : {:.4}'.format(rmse_target))
print('gradient boost all : {:.4}'.format(rmse_all))
print('gradient boost source : {:.4}'.format(rmse_source))

gradient boost target : 8.436
gradient boost all : 9.999
gradient boost source : 13.72


# 学習済みモデルの取り出し

In [12]:
print(len(reg_target.estimators_))
print(len(reg_all.estimators_))
print(len(reg_source.estimators_))

30
200
150


In [13]:
reg_source.estimators_

array([[DecisionTreeRegressor(criterion='friedman_mse', max_depth=4,
                      random_state=RandomState(MT19937) at 0x20BAFC77840)],
       [DecisionTreeRegressor(criterion='friedman_mse', max_depth=4,
                      random_state=RandomState(MT19937) at 0x20BAFC77840)],
       [DecisionTreeRegressor(criterion='friedman_mse', max_depth=4,
                      random_state=RandomState(MT19937) at 0x20BAFC77840)],
       [DecisionTreeRegressor(criterion='friedman_mse', max_depth=4,
                      random_state=RandomState(MT19937) at 0x20BAFC77840)],
       [DecisionTreeRegressor(criterion='friedman_mse', max_depth=4,
                      random_state=RandomState(MT19937) at 0x20BAFC77840)],
       [DecisionTreeRegressor(criterion='friedman_mse', max_depth=4,
                      random_state=RandomState(MT19937) at 0x20BAFC77840)],
       [DecisionTreeRegressor(criterion='friedman_mse', max_depth=4,
                      random_state=RandomState(MT19937) at 0x

In [14]:
idx = 1
model = reg_source.estimators_[idx][0]
pred_idx = model.predict(target_test_X)
pred_idx

array([-19.55432342,   4.21026024, -19.55432342,   4.21026024,
         4.21026024,   4.21026024, -19.55432342, -19.55432342,
         4.21026024,   4.21026024, -19.55432342, -19.55432342,
         4.21026024,   4.21026024, -19.55432342,   4.21026024,
         4.21026024,   4.21026024,   4.21026024, -19.55432342,
       -19.55432342,   4.21026024,   4.21026024,  -6.0720375 ,
        -6.0720375 ,   4.21026024, -19.55432342,   4.21026024,
        14.79359624,   4.21026024,  14.79359624,   4.21026024,
         4.21026024, -19.55432342,   4.21026024, -19.55432342,
         4.21026024,   4.21026024,   4.21026024,   4.21026024,
         4.21026024,   4.21026024, -19.55432342,   4.21026024,
       -19.55432342,  14.79359624,  14.79359624,   4.21026024,
         4.21026024,   4.21026024,   6.05414475,  21.169268  ,
         4.21026024,   6.05414475,  -6.0720375 ,   4.21026024,
        -6.0720375 ,  -6.0720375 ,  -6.0720375 ,   4.21026024,
        21.169268  ,  21.169268  ,   4.21026024,   4.21

In [15]:
diff = y_pred_source - target_test_y
diff

array([ 2.66971336e+00,  3.78615896e+00,  3.16658383e+00,  1.06166219e+01,
        1.54934121e+01,  1.06758918e+01,  1.04376039e+01,  1.14867832e+01,
        5.71041382e+00,  1.26652374e+01,  1.32823456e+00,  2.57812903e+00,
        6.81444414e+00,  1.01188780e+01,  4.82123151e+00,  1.93620364e+01,
        1.93520364e+01,  1.17584723e+01,  1.06334365e+01,  3.77896769e+00,
        3.47808953e+00,  1.09468294e+01,  1.30368294e+01,  1.70228632e+01,
        1.70228632e+01,  2.56320636e+01,  6.34342997e+00,  1.62546453e+01,
        2.36676481e+01,  2.10597886e+01,  2.55276078e+01,  6.05786050e+00,
        8.38210700e+00,  3.22948210e-01,  2.08646186e+00, -8.73060355e-01,
        2.78215867e+00,  1.85622972e+01,  4.23893669e-01,  1.85622972e+01,
        4.23893669e-01,  7.53762364e+00,  6.67429434e+00,  5.61266446e+00,
        6.06293100e+00,  1.55145213e+01,  1.55145213e+01,  1.48725349e+01,
        1.48625349e+01,  1.68192369e+01,  3.34850938e+01,  3.16678624e+01,
        1.68292369e+01,  

In [16]:
np.sum(pred_idx * diff)

13085.659818219097

In [17]:
diffs = np.empty(len(reg_source.estimators_), float)
for idx, estimator in enumerate(reg_source.estimators_):
    model = estimator[0]
    pred_idx = model.predict(target_test_X)
    diffs[idx] = np.sum(pred_idx * diff)

In [18]:
diffs

array([ 2.37469652e+04,  1.30856598e+04,  9.85942957e+03,  1.77297113e+04,
        1.21063621e+04,  1.93828121e+04,  5.68198177e+03,  1.90278210e+03,
        1.37509738e+04, -1.13879054e+03,  2.70286841e+03,  1.41136527e+03,
        1.21787550e+04, -1.63737847e+03,  6.21873140e+03,  1.06442105e+02,
        5.68829308e+02, -2.79111704e+03,  1.55270930e+03, -4.19585982e+02,
       -1.13907600e+03,  1.27631968e+03,  2.81457338e+02, -1.98174732e+03,
       -9.38377010e+02, -2.11311710e+02,  2.14266914e+03,  2.55793942e+03,
       -1.29938079e+03, -2.53363698e+02, -5.95047661e+02, -3.82343769e+03,
        4.45745023e+01,  3.05598059e+02, -1.81923173e+03,  7.72972739e+01,
        2.72652749e+02,  3.88306913e+02, -3.12246548e+03,  4.28041862e+03,
       -1.01189598e+03,  1.15695864e+03,  2.72513560e+01,  3.78988195e+03,
       -1.37233579e+03, -1.67346350e+03, -1.69783470e+02,  9.01837674e+02,
        1.37864534e+03, -2.42190104e+03, -1.45500591e+03,  1.30267502e+03,
       -2.34483209e+03, -

In [19]:
reg_source.fit_domain_adaptation(target_train_X, target_train_y, drop_rate=0.01)

150
150


GradientBoostingRegressor(max_depth=4, n_estimators=160, random_state=0)

In [20]:
len(reg_source.estimators_)

160

In [21]:
pred = reg_source.predict_domain_adaptation(target_test_X)

In [22]:
rmse_dart = mean_squared_error(target_test_y, pred)**0.5
print('gradient boost dart : {:.4}'.format(rmse_dart))

gradient boost dart : 12.79


# early stoppingあり

In [23]:
target_domain = 0

target_train = df.query('label==@target_domain').sample(n_target)
target_test = df.query('label==@target_domain').drop(index=target_train.index)
target_train_X = target_train.drop(columns=[y_feature, 'label'])
target_train_y = target_train[y_feature]
target_test_X = target_test.drop(columns=[y_feature, 'label']).values
target_test_y = target_test[y_feature].values
source_X = df.query('label!=@target_domain').drop(columns=[y_feature, 'label'])
source_y = df.query('label!=@target_domain')[y_feature]

X_train = pd.concat([source_X, target_train_X]).values
y_train = pd.concat([source_y, target_train_y]).values
sample_size = [source_X.shape[0], target_train_X.shape[0]]


# targetのみgradient boost
reg_target = GradientBoostingRegressor(
                    n_estimators = 1000, 
                    learning_rate=learning_rate,
                    max_depth = max_depth,
                    validation_fraction = validation_fraction,
                    n_iter_no_change= n_iter_no_change,
                    tol= tol,
                    random_state = random_state)
reg_target.fit(target_train_X.values, target_train_y.values)
y_pred_target = reg_target.predict(target_test_X)
rmse_target = mean_squared_error(target_test_y, y_pred_target)**0.5

# source+target gradient boost
reg_all = GradientBoostingRegressor(
                    n_estimators = 1000, 
                    learning_rate=learning_rate,
                    max_depth = max_depth,
                    validation_fraction = validation_fraction,
                    n_iter_no_change= n_iter_no_change,
                    tol= tol,
                    random_state = random_state)
reg_all.fit(X_train, y_train)
y_pred_all = reg_all.predict(target_test_X)
rmse_all = mean_squared_error(target_test_y, y_pred_all)**0.5

# sourceのみgradient boost
reg_source = GradientBoostingRegressor(
                    n_estimators = 1000, 
                    learning_rate=learning_rate,
                    max_depth = max_depth,
                    validation_fraction = validation_fraction,
                    n_iter_no_change= n_iter_no_change,
                    tol= tol,
                    random_state = random_state)
reg_source.fit(source_X, source_y)
y_pred_source = reg_source.predict(target_test_X)
rmse_source = mean_squared_error(target_test_y, y_pred_source)**0.5

print('gradient boost target : {:.4}'.format(rmse_target))
print('gradient boost all : {:.4}'.format(rmse_all))
print('gradient boost source : {:.4}'.format(rmse_source))

gradient boost target : 7.791
gradient boost all : 11.0
gradient boost source : 15.0


In [24]:
print(len(reg_target.estimators_))
print(len(reg_all.estimators_))
print(len(reg_source.estimators_))

31
124
155


In [73]:
from copy import deepcopy

reg_dart = deepcopy(reg_source)
reg_dart.fit_domain_adaptation(target_train_X, target_train_y, drop_rate=0.1, n_estimators_add=10)

155
155


GradientBoostingRegressor(max_depth=4, n_estimators=165, random_state=0,
                          tol=0.1, validation_fraction=0.2)

In [74]:
pred = reg_dart.predict_domain_adaptation(target_test_X)
rmse_dart = mean_squared_error(target_test_y, pred)**0.5
print('gradient boost dart : {:.4}'.format(rmse_dart))

gradient boost dart : 11.65


In [27]:
print(len(reg_source.estimators_))
print(len(reg_dart.estimators_))

155
170


In [28]:
print(reg_source.n_estimators)
print(reg_source.n_estimators_)

1000
155
